In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))


In [3]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [4]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['themes'].to_list())

In [5]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [6]:
embedding_dim = 64

model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(500, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(len(Y[0]), activation='sigmoid')
])

model1.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5

In [20]:
epochs = 5
model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs)

Epoch 1/10
385/385 [==============================] - 85s 220ms/step - loss: 10.6270 - categorical_accuracy: 0.3263 - val_loss: 23.0920 - val_categorical_accuracy: 0.0767
Epoch 2/10
385/385 [==============================] - 85s 221ms/step - loss: 10.0911 - categorical_accuracy: 0.3383 - val_loss: 24.9509 - val_categorical_accuracy: 0.0723
Epoch 3/10
385/385 [==============================] - 85s 221ms/step - loss: 9.7673 - categorical_accuracy: 0.3449 - val_loss: 24.8936 - val_categorical_accuracy: 0.0723
Epoch 4/10
385/385 [==============================] - 85s 222ms/step - loss: 9.5486 - categorical_accuracy: 0.3514 - val_loss: 26.8662 - val_categorical_accuracy: 0.0548
Epoch 5/10
385/385 [==============================] - 85s 220ms/step - loss: 9.3878 - categorical_accuracy: 0.3426 - val_loss: 26.7485 - val_categorical_accuracy: 0.0592
Epoch 6/10
385/385 [==============================] - 85s 221ms/step - loss: 9.2717 - categorical_accuracy: 0.3522 - val_loss: 27.2393 - val_categor

In [21]:
score = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

43/43 [==============================] - 1s 15ms/step - loss: 28.2201 - categorical_accuracy: 0.0643
Test Score: 28.220096588134766
Test Accuracy: 0.06428049504756927


In [9]:
'''
class_names = ['out of cat ', 'in cat']
cm = confusion_matrix(Y_test, np.rint(y_pred))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot()
plt.show()'''

"\nclass_names = ['out of cat ', 'in cat']\ncm = confusion_matrix(Y_test, np.rint(y_pred))\ndisp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)\ndisp.plot()\nplt.show()"

### Model 2 LSTM

In [10]:
embedding_dim =100 

model2 = keras.Sequential([
    layers.Embedding(max_features, embedding_dim, input_length=max_len),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(len(Y[0]), activation='sigmoid')
])

In [11]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 99)                9999      
Total params: 1,090,399
Trainable params: 1,090,399
Non-trainable params: 0
_________________________________________________________________


In [13]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string
batch_size = 128
epochs = 5

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[early_stopping_callback])

Epoch 1/5
87/87 [==============================] - 17s 195ms/step - loss: 19.0051 - categorical_accuracy: 0.0032 - val_loss: 18.8436 - val_categorical_accuracy: 0.0032
Epoch 2/5
87/87 [==============================] - 17s 193ms/step - loss: 18.4259 - categorical_accuracy: 0.0017 - val_loss: 18.8219 - val_categorical_accuracy: 0.0032
Epoch 3/5
87/87 [==============================] - 17s 192ms/step - loss: 18.4121 - categorical_accuracy: 0.0017 - val_loss: 18.8022 - val_categorical_accuracy: 0.0032
Epoch 4/5
87/87 [==============================] - 17s 191ms/step - loss: 18.3958 - categorical_accuracy: 0.0017 - val_loss: 18.7963 - val_categorical_accuracy: 0.0032
Epoch 5/5
87/87 [==============================] - 17s 190ms/step - loss: 18.3771 - categorical_accuracy: 0.0017 - val_loss: 18.8038 - val_categorical_accuracy: 0.0032


In [14]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

43/43 [==============================] - 1s 15ms/step - loss: 18.5577 - categorical_accuracy: 0.0022
Test set
  Loss: 18.558
  Accuracy: 0.002


In [15]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [17]:
batch_size = 64
epochs = 5
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
174/174 [==============================] - 5s 31ms/step - loss: 601521.3750 - accuracy: 0.0203 - val_loss: 3190682.5000 - val_accuracy: 0.0032
Epoch 2/5
174/174 [==============================] - 5s 29ms/step - loss: 55591072.0000 - accuracy: 0.0226 - val_loss: 164269184.0000 - val_accuracy: 0.0032
Epoch 3/5
174/174 [==============================] - 5s 29ms/step - loss: 633631616.0000 - accuracy: 0.0211 - val_loss: 1337953024.0000 - val_accuracy: 0.0032
Epoch 4/5
174/174 [==============================] - 5s 29ms/step - loss: 3085816064.0000 - accuracy: 0.0232 - val_loss: 5515577856.0000 - val_accuracy: 0.0170
Epoch 5/5
174/174 [==============================] - 5s 29ms/step - loss: 9826993152.0000 - accuracy: 0.0227 - val_loss: 15176954880.0000 - val_accuracy: 0.0032


In [18]:
accr = model3.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

43/43 [==============================] - 0s 4ms/step - loss: 15214229504.0000 - accuracy: 0.0022
Test set
  Loss: 15214229504.000
  Accuracy: 0.002


In [19]:
len(df.themes[0])

99